# chapter 4. Classification

## Ensemble

**앙상블 학습을 통한 분류**는 **여러 개의 분류기를 생성하고 그 예측을 결합함으로써 보다 정확한 최종 예측을 도출하는 기법**을 말한다. <br>
**정형 데이터 분류시에는 앙상블이 뛰어난 성능**을 나타내고 있다. <br>
**랜덤 포레스트**와 **그래디언트 부스팅 알고리즘**은 뛰어난 성능과 쉬운 사용, 다양한 활용도로 인해 그간 분석가 및 데이터 과학자들 사이에서 많이 애용되었다. <br> 

앙상블 학습 유형은 전통적으로 **보팅(Voting), 배깅(Bagging), 부스팅(Boosting)** 의 세가지로 나눌 수 있다. <br>
**보팅과 배깅은** 여러 개의 분류기가 **투표를 통해 최종 예측 결과를 결정**하는 방식이다. <br>
보팅과 배깅의 다른 점은 보팅의 경우 일반적으로 서로 다른 알고리즘을 가진 분류기를 결합하는 것이고, 배깅의 경우 각각의 분류기가 모두 같은 유형의 알고리즘 기반이지만, 데이터 샘플링을 서로 다르게 가져가면서 학습을 수행해 보팅을 수행해야 하는 것이다. <br>
**대표적인 배깅 방식이 바로 랜덤 포레스트 알고리즘**이다. <br>

<img src = 'image/Voting and Bagging.jpg' alt = 'Voting and Bagging' width='600' height='600'>

왼쪽 그림은 **보팅 분류기**를 도식화한 것이고, 오른쪽 그림은 **배팅 분류기**를 도식화한 것이다. <br>

**개별 Classifier에 데이터를 샘플링해서 추출하는 방식을 부스트트래핑(Boosttrapping) 분할 방식**이라고 부른다. <br>
개별 분류기가 부스트트래핑 방식으로 샘플링 된 데이터 세트에 대해서 학습을 통해 개별적인 예측을 수행한 결과를 보팅을 통해서 최종 예측 결과를 선정하는 방식이 바로 **배깅 앙상블 방식**이다. <br>
배깅 방식은 중첩을 허용한다. <br>

부스팅은 여러 개의 분류기가 순차적으로 학습을 수행하되, 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서는 올바르게 예측할 수 있도록 다름 분류기에게는 가중치를 부여하면서 학습과 예측을 진행한다.

### 하드 보팅(Hard Voting)과 소프트 보팅(Soft Voting)

보팅 방법은 **하드 보팅**과  **소프트 보팅** 두 가지가 있다. <br>
**하드 보팅**을 이용한 분류는 **예측한 결과값들 중 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정**하는 것이다. <br>
**소프트 보팅**은 **분류기들의 레이블 값 결정 확률을 모두 더하고 이를 평균해서 이들 중 가장 높은 레이블 값을 최종 보팅 결과값으로 선정**한다. <br>

<img src = 'image/Hard Voting and Soft Voting.jpg' alt = 'Hard Voting and Soft Voting' width='600' height='600'>

### 보팅 분류기(Voting Classifier)

사이킷런은 보팅 방식의 앙상블을 구현한 **Voting Classifier 클래스**를 제공한다. 

In [8]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns = cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


**VotingClassifier 클래스**는 주로 **생성 인자로 estimators와 voting 값을 입력받는다.** <br>
**estimators**는 리스트 값으로 보팅에 사용될 여러 개의 Classifier 객체들을 튜플 형식으로 입력받으며 **voting**은 hard 시 하드 보팅, soft 시 소프트 보팅을 적용하라는 의미이다. 

In [10]:
# 개별 모델은 로지스틱 회귀와 KNN이다. 
lr_clf = LogisticRegression(solver = 'liblinear')
knn_clf = KNeighborsClassifier(n_neighbors = 8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators = [('LR', lr_clf), ('KNN', knn_clf)], voting = 'soft')

X_train, X_test, Y_train, Y_test = train_test_split(cancer.data, cancer.target, 
                                                   test_size = 0.2, random_state = 156)

# VotingClassifier 학습 / 예측 / 평가
vo_clf.fit(X_train, Y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도 : {0:.4f}'.format(accuracy_score(Y_test, pred)))

# 개별 모델의 학습 / 예측 / 평가
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, Y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print('{0} 정확도 : {1:.4f}'.format(class_name, accuracy_score(Y_test, pred)))

Voting 분류기 정확도 : 0.9561
LogisticRegression 정확도 : 0.9474
KNeighborsClassifier 정확도 : 0.9386


보팅으로 여러 개의 기반 분류기를 결합한다고 해서 무조건 기반 분류기보다 예측 성능이 향상되지는 않는다. <br>
다양한 요건에 따라 오히려 기반 분류기 중 가장 좋은 분류기의 성능이 보팅했을 때보다 나을 수도 있다. <br>
그럼에도 불구하고 보팅을 포함한 배깅과 부스팅의 앙상블 방법은 단일 ML 알고리즘보다 뛰어난 예측 성능을 가지는 경우가 많다. <br>

**ML 모델의 성능은 이렇게 다양항 테스트 데이터에 의해 검증되므로 어떻게 높은 유연성을 가지고 현실에 대처할 수 있는가가 중요한 ML 모델의 평가 요소가 된다.** <br>
편향-분산 트레이드오프는 ML 모델이 극복해야 할 중요한 과제이다. <br>

**배깅과 부스팅은 대부분 결정 트리 알고리즘을 기반**으로 한다. <br>
앙상블 학습에서는 결정 트리 알고리즘의 단점을 수십-수천개의 매우 많은 분류기를 결합해 다양한 상황을 학습하게 함으로써 극복하고 있다. <br>
**결정 트리 알고리즘의 장점은 그대로 취하고 단점은 보완하면서 편향-분산 트레이드오프의 효과를 극대화할 수 있다는 것**이다. 